# 6-1. 프로젝트: CutMix 또는 Mixup 비교실험 하기

지금까지 기본적인 augmentation 방법을 적용해 모델을 훈련시키고, 최신 augmentation 기법을 배워 보았습니다.

이번에는 최신 기법(CutMix 또는 Mixup)을 적용해 모델을 훈련시켜 봅시다. 데이터셋에 두 가지 방법 중 하나를 적용하고, 모델을 학습시켜 주세요. 결과를 수치화하고 비교하는 것도 잊지 마세요!

우선 주요 라이브러리 버전을 확인해 볼까요?






In [2]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)


2.16.2
1.26.4


In [3]:
# 1. Augmentation을 적용한 데이터셋 만들기
# 이전 스텝과 아래 코드를 참고하여 데이터셋에 CutMix 또는 Mixup augmentation을 적용해 주세요.


def normalize_and_resize_img(image, label):
    # Normalizes images: `uint8` -> `float32`
    image = tf.image.resize(image, [224, 224])
    return tf.cast(image, tf.float32) / 255.0, label


def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.clip_by_value(image, 0, 1)
    return image, label


def onehot(image, label):
    label = tf.one_hot(label, num_classes)
    return image, label


def apply_normalize_on_dataset(
    ds, is_test=False, batch_size=16, with_aug=False, with_cutmix=False
):
    ds = ds.map(normalize_and_resize_img, num_parallel_calls=2)
    if not is_test and with_aug:
        ds = ds.map(augment)
    ds = ds.batch(batch_size)
    if not is_test and with_cutmix:
        ds = ds.map(cutmix, num_parallel_calls=2)
    else:
        ds = ds.map(onehot, num_parallel_calls=2)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds


print("=3")


=3
